In [ ]:
# connect google drive
from google.colab import drive 
drive.mount('/content/drive')
# Download konlpy
%cd ./drive/MyDrive/Colab\ Notebooks/
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git 
%cd ./Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab_light_220429.sh

In [2]:
from konlpy.tag import Mecab
mecab = Mecab()

sentence = "실용자연어처리 실습 진행중 입니다."
print(mecab.morphs(sentence)) #형태소 분석
print(mecab.nouns(sentence)) #체언 분석(명사도다 체언이 정확!)
print(mecab.pos(sentence)) #형태소와 형태소 태그 분석
#mecab 품사 태그 검색하면 됨 http://openuiz.blogspot.com/2016/07/mecab-ko-dic.html
#N으로 시작하면 명사 , J로 시작하면 조사더라~ 

['실용', '자연어', '처리', '실습', '진행', '중', '입니다', '.']
['실용', '자연어', '처리', '실습', '진행', '중']
[('실용', 'NNG'), ('자연어', 'NNG'), ('처리', 'NNG'), ('실습', 'NNG'), ('진행', 'NNG'), ('중', 'NNB'), ('입니다', 'VCP+EF'), ('.', 'SF')]


In [ ]:
!pip install datasets

!를 쓰는 것은 쉘의 언어라고 한다. 
파이썬 코드가 아니라 쉘 명령어야 데이터 셋을 다운로드 해줘!



- 데이터 구하기 단계

In [ ]:
from datasets import load_dataset #Huggingface 데이터셋 패키지 import
data = load_dataset("sepidmnorozy/Korean_sentiment") # 데이터 다운로드

In [5]:
print("Data type: ", type(data)) #데이터 타입 확인
print("Data structure: ", data) #데이터 구조 확인
print("Data keys: ", data.keys()) #데이터 키 확인

print(data['train'][0]) #데이터 타입 확인

Data type:  <class 'datasets.dataset_dict.DatasetDict'>
Data structure:  DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 36000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 2667
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1333
    })
})
Data keys:  dict_keys(['train', 'test', 'validation'])
{'label': 1, 'text': '역시 명작 어렸을때 봤을때도 재밌었고 지금 봐도 몇억배 이상으로 재밌어요'}


- 라벨이 1인 이유는 평점이 긍정이기 때문에 준 것으로 보인다.
- 라벨이 0이면 평점이 부정적!

In [6]:
train_data = data['train']
dev_data = data['validation']
test_data = data['test']

print(train_data)
print(dev_data)
print(test_data)

#형태소 단위로 Tokenization 된 텍스트를 {train/dev/test}_data에 저장
train_data = train_data.map(lambda example: {'label': example['label'], 'text': ' '.join(mecab.morphs(example['text']))})
dev_data = dev_data.map(lambda example: {'label': example['label'], 'text': ' '.join(mecab.morphs(example['text']))})
test_data = test_data.map(lambda example: {'label': example['label'], 'text': ' '.join(mecab.morphs(example['text']))})
# example은 본인 자체를 나타낸다. lambda라는 거는 함수인데 런타임 도중 함수 실행하는 거라고 한다. 그래서 의미는 label은 원래 label 그대로 넣고, text 데이터는 형태소 분석한 텍스트로 대체한다.
#조인을 하면 단어 사이에 ' ' 띄어쓰기를 넣겠다라는 것
#역시 명작 어렸을 때 ==> 역시 명작 어렸 을 때 이렇게 형태소 단위로 띄어버린다.

Dataset({
    features: ['label', 'text'],
    num_rows: 36000
})
Dataset({
    features: ['label', 'text'],
    num_rows: 1333
})
Dataset({
    features: ['label', 'text'],
    num_rows: 2667
})


Map:   0%|          | 0/36000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1333 [00:00<?, ? examples/s]

Map:   0%|          | 0/2667 [00:00<?, ? examples/s]

데이터 탐색 시각화 단계는 패스!! - 전처리를 하지 않아서 아래 과정은 하지 않아도 된다고 한다.

- 데이터 준비 단계

In [31]:
#### 텍스트와 범주형 특성 다루기 (Bag-of-words 방식으로 벡터화 하는 법 구현)
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer # CountVectorizer : Bag of words 벡터화 하기 위한 클래스

# ko_stop_words = ['은', '는', '이', '가', '께서', '에서']
# ko_stop_words = ['때', '었', '고']
ko_stop_words = ['은', '는', '이', '가', '께서', '에서', '부터', '까지', '에게', '한테', '와', '과', '을', '를', '의', '로서', '로', '그리고', '즉', '게다가', '그런데', '따라서', '그러나', '그래도', '오히려', '비록', '또는', '및', '고', '어', '으려고', '지만', '다', '는다', '냐', '으냐', '요', '어요', '때', '사람', '영화', '강아지', '컴퓨터', '언어']

#데이터를 벡터화 해주는 모델
vectorizer = CountVectorizer(strip_accents='unicode', token_pattern=r"(?u)\b\w\w+\b|'\w+", ngram_range=(1,1)) # 기준 모델 - 80.16  #한국어 소실되지 않게 5주차에 추가됨!
# vectorizer = CountVectorizer(strip_accents='unicode', token_pattern=r"(?u)\b\w\w+\b|'\w+", ngram_range=(1,2)) # 모델 2.1
# vectorizer = CountVectorizer(strip_accents='unicode', token_pattern=r"(?u)\b\w\w+\b|'\w+", ngram_range=(1,3)) # 모델 2.2 #ngram은 uni~trigram까지 만들겠다는 뜻
# vectorizer = CountVectorizer(strip_accents='unicode', token_pattern=r"(?u)\b\w\w+\b|'\w+", ngram_range=(1,5)) # 모델 2.3 # 오히려 성능이 저하되더라.
# vectorizer = TfidfVectorizer(strip_accents='unicode', token_pattern=r"(?u)\b\w\w+\b|'\w+") # 모델 3 #데이터를 벡터화 해주는 모델 # TF-IDF만 적용한 모델
# vectorizer = TfidfVectorizer(strip_accents='unicode', token_pattern=r"(?u)\b\w\w+\b|'\w+", ngram_range=(1,2)) # 모델 4 
# vectorizer = CountVectorizer(strip_accents='unicode', token_pattern=r"(?u)\b\w\w+\b|'\w+") # 모델 5 형태소 분석 적용 시작
# vectorizer = CountVectorizer(stop_words=ko_stop_words) #                                     모델 6 stop words(불용어) 처리
# vectorizer = CountVectorizer(strip_accents='unicode', token_pattern=r"(?u)\b\w\w+\b|'\w+", ngram_range=(1,1)) # 모델 7==모델 5, 형태소 O
# vectorizer = CountVectorizer(strip_accents='unicode', token_pattern=r"(?u)\b\w\w+\b|'\w+", ngram_range=(1,2)) # 모델 8 형태소 O
# vectorizer = CountVectorizer(strip_accents='unicode', token_pattern=r"(?u)\b\w\w+\b|'\w+", ngram_range=(1,3)) # 모델 9 형태소 O
# vectorizer = TfidfVectorizer(strip_accents='unicode', token_pattern=r"(?u)\b\w\w+\b|'\w+", ngram_range=(1,1)) # 모델 10 형태소 X
# vectorizer = TfidfVectorizer(strip_accents='unicode', token_pattern=r"(?u)\b\w\w+\b|'\w+", ngram_range=(1,2)) # 모델 11 형태소 X
# vectorizer = TfidfVectorizer(strip_accents='unicode', token_pattern=r"(?u)\b\w\w+\b|'\w+", ngram_range=(1,3)) # 모델 12 형태소 X
# vectorizer = TfidfVectorizer(strip_accents='unicode', token_pattern=r"(?u)\b\w\w+\b|'\w+", ngram_range=(1,1)) # 모델 13 형태소 O
# vectorizer = TfidfVectorizer(strip_accents='unicode', token_pattern=r"(?u)\b\w\w+\b|'\w+", ngram_range=(1,2)) # 모델 14 형태소 O
# vectorizer = TfidfVectorizer(strip_accents='unicode', token_pattern=r"(?u)\b\w\w+\b|'\w+", ngram_range=(1,3)) # 모델 15 형태소 O
# vectorizer = TfidfVectorizer(stop_words=ko_stop_words, ngram_range=(1,2)) # 모델 16 불용어 처리 및 형태소 분석
vectorizer.fit(train_data['text']) # 모델을 학습 시키는 것, 텍스트 문서 모음을 토큰 수의 행렬로 변환 => 문자 -> 수치화

# print(vectorizer.vocabulary_) # 어휘 집합을 출력 --> 너무 길어 주석 처리로 변경
print("어휘 집합 길이 : ", len(vectorizer.vocabulary_)) # 어휘 집합의 길이 출력
#형태소 분석을 하니 토큰 수가 100,494에서 26,879로 바뀌더라
#형태소 분석 전에는 어절로 분석했다. 봤을 때도, 먹었을 때도 ==> 봤을 때 도, 먹었 을 때 도 == 토큰의 개수가 확 줄었다.
#이전에는 먹었을때도 먹었을때도 봤을 때도를 아예 다른 토큰으로 만든다. 봤을지라도, 봤을건데 전부다 다른 토큰 수로 추가해야 한다.
#근데 봤을 때 도 하면은 토큰에 추가해야 하는 단어가 줄어든다. 단어들이 더 쪼개 지면서 재사용이 될 수 있어짐. 전체 토큰이 10만개에서 2만개로 줄어든 것이다!! 이해 되었다!

train_vectors = vectorizer.transform(train_data['text'])
dev_vectors = vectorizer.transform(dev_data['text'])
test_vectors = vectorizer.transform(test_data['text'])

어휘 집합 길이 :  26879


형태소 분석을 하니 토큰 수가 100,494에서 26,879로 바뀌더라
형태소 분석 전에는 어절로 분석했다.

 봤을 때도, 먹었을 때도 ==> 봤을 때 도, 먹었 을 때 도 == 토큰의 개수가 확 줄었다.


이전에는 먹었을때도 먹었을때도 봤을 때도를 아예 다른 토큰으로 만든다. 봤을지라도, 봤을건데 전부다 다른 토큰 수로 추가해야 한다.


근데 봤을 때 도 하면은 토큰에 추가해야 하는 단어가 줄어든다. 단어들이 더 쪼개 지면서 재사용이 될 수 있어짐. 전체 토큰이 10만개에서 2만개로 줄어든 것이다!! 이해 되었다!


In [32]:
print(train_data[0]['text']) # 결과 확인
print("---------------------")
print(train_vectors) # 결과 확인

역시 명작 어렸 을 때 봤 을 때 도 재밌 었 고 지금 봐도 몇 억 배 이상 으로 재밌 어요
---------------------
  (0, 2205)	1
  (0, 6341)	1
  (0, 7257)	2
  (0, 9070)	1
  (0, 9086)	1
  (0, 10468)	1
  (0, 11201)	1
  (0, 11281)	1
  (0, 15882)	1
  (0, 15953)	1
  (0, 16039)	1
  (0, 16182)	1
  (0, 16504)	1
  (0, 18227)	1
  (0, 18328)	2
  (0, 18706)	1
  (0, 19803)	2
  (0, 21267)	1
  (1, 5370)	1
  (1, 7550)	1
  (1, 8490)	1
  (1, 8538)	1
  (1, 15766)	1
  (1, 16572)	1
  (1, 16757)	1
  :	:
  (35997, 17247)	1
  (35997, 18458)	1
  (35997, 19116)	1
  (35997, 20826)	1
  (35998, 146)	1
  (35998, 3939)	1
  (35998, 4002)	1
  (35998, 4985)	1
  (35998, 6341)	1
  (35998, 7540)	1
  (35998, 13867)	1
  (35998, 18471)	1
  (35998, 20000)	1
  (35998, 20260)	1
  (35998, 21581)	2
  (35998, 23405)	1
  (35999, 5370)	1
  (35999, 6697)	1
  (35999, 6996)	1
  (35999, 8459)	1
  (35999, 12463)	1
  (35999, 16173)	1
  (35999, 16526)	1
  (35999, 18426)	1
  (35999, 23358)	1


왼쪽에 35999는 train 데이터가 36000개였기 때문이다.
train 36000번째 값은 개수가 35999가 4개 이므로 4개라고 볼 수 있다. 확인해보자 오잉 왜 아니지 싸이킷런 내부 문제 -> 5주차에 해결됨!

In [33]:
print(train_data[35999]['text'])

큰 의미 없이 연결 ' 만 ' 되 는 살인 들 .


In [34]:
print(train_data['label'])

[1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 

In [35]:
sample_num = 0 #확인하고 싶은 샘플 번호
sample_origin = train_data[sample_num] #확인하고 싶은 샘플의 원래 문장
print("Original Input: ", sample_origin) 
print("------------------------------------------------------")
sample_transform = train_vectors[sample_num]
print(sample_transform) # 확인하고 싶은 샘플의 transform된 결과
#CountVectorizer는 count만 나오는데 tfdif방식은 가중치를 볼 수 있더라 tf-idf의 로그 계산 한 거 나오는 것
print("------------------------------------------------------")
sample_inverse_transfrom = vectorizer.inverse_transform(sample_transform) # 확인하고 싶은 샘플의 transform된 결과를 다시 단어의 조합으로 바꾼 결과
print(sample_inverse_transfrom)

Original Input:  {'label': 1, 'text': '역시 명작 어렸 을 때 봤 을 때 도 재밌 었 고 지금 봐도 몇 억 배 이상 으로 재밌 어요'}
------------------------------------------------------
  (0, 2205)	1
  (0, 6341)	1
  (0, 7257)	2
  (0, 9070)	1
  (0, 9086)	1
  (0, 10468)	1
  (0, 11201)	1
  (0, 11281)	1
  (0, 15882)	1
  (0, 15953)	1
  (0, 16039)	1
  (0, 16182)	1
  (0, 16504)	1
  (0, 18227)	1
  (0, 18328)	2
  (0, 18706)	1
  (0, 19803)	2
  (0, 21267)	1
------------------------------------------------------
[array(['고', '도', '때', '명작', '몇', '배', '봐도', '봤', '어렸',
       '어요', '억', '었', '역시', '으로', '을', '이상', '재밌',
       '지금'], dtype='<U62')]


In [36]:
print(train_data[-1])
print(train_vectors[-1])
print(vectorizer.inverse_transform(train_vectors[-1]))
# 5주차에서 unicode 넣고 단어 사라짐 문제 해결!!

{'label': 0, 'text': "큰 의미 없이 연결 ' 만 ' 되 는 살인 들 ."}
  (0, 5370)	1
  (0, 6697)	1
  (0, 6996)	1
  (0, 8459)	1
  (0, 12463)	1
  (0, 16173)	1
  (0, 16526)	1
  (0, 18426)	1
  (0, 23358)	1
[array(['는', '되', '들', '만', '살인', '없이', '연결', '의미',
       '큰'], dtype='<U62')]


- 선택 훈련 단계

In [37]:
from sklearn.svm import LinearSVC # LinearSVC: linear SVM을 사용하기 위한 클래스
svm = LinearSVC() # 모델 정의
svm.fit(train_vectors, train_data['label']) # 지도 학습 진행 정답 레이블과 함께 훈련 진행.

print(svm.coef_) # weights (w)
print("--------------------")
print(svm.intercept_)  # bias (b)

[[-0.05649397  0.          0.         ...  0.34662051  0.12389057
   0.        ]]
--------------------
[-0.13878212]


/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


fit을 할 때는 데이터와 label를 넣어줘야한다.

- 교차 검증을 사용한 평가

In [38]:
from sklearn.model_selection import cross_val_score

all_data = train_data['text']+dev_data['text']+test_data['text'] # all_data = train_data + dev_data + test_data 의 text 다 합쳤다.
all_label = train_data['label']+dev_data['label']+test_data['label'] # all_label은 train_data, dev_data, test_date의 레이블을 모두 합한 것
all_vectors = vectorizer.transform(all_data) # all_data(텍스트)를 숫자로 변환 (벡터화) 그걸 올 벡터
scores = cross_val_score(svm, all_vectors, all_label, cv=5) # 5-cross validation - cv는 몇 번 할거냐 5개로 데이터 나누고 1개 테스트로 시킨다. 이게 커질수록 학습이 오래 걸린다.

print("All scores: ", scores) # 전체 점수 출력
print("Average: {: .2f}%".format(scores.mean()*100)) # 다섯 번의 스코어 평균 출력
print("Standard deviation: {:.6f}".format(scores.std())) # 표준편차 출력

/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


All scores:  [0.81225  0.808    0.807625 0.80925  0.815   ]
Average:  81.04%
Standard deviation: 0.002806


/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
scores = cross_val_score(svm, all_vectors, all_label, cv=10) # 10-cross validation - cv는 몇 번 할거냐 10개로 데이터 나누고 1개 테스트로 시킨다. 이게 커질수록 학습이 오래 걸린다.

In [40]:
print("All scores: ", scores) # 전체 점수 출력
print("Average: {: .2f}%".format(scores.mean()*100)) # 다섯 번의 스코어 평균 출력
print("Standard deviation: {:.6f}".format(scores.std())) # 표준편차 출력

All scores:  [0.8165  0.80775 0.80925 0.8135  0.814   0.80475 0.81975 0.807   0.81475
 0.817  ]
Average:  81.24%
Standard deviation: 0.004696


왜 교수님께서는 77.44%라고 하셨나?? (맨처음한 것 내가 뭔가 잘못했었던 듯)

In [41]:
# from sklearn.model_selection import GridSearchCV
# param_grid = [{'max_iter': [500, 1000, 5000], 'C': [1, 10, 100]}] # 학습 max_iter와 C를 변동 하이퍼파라미터로 설정
# # 500이고 1일때, 1000이고 10일 때 5000이고 100일 때
# # Regularization parameter 그냥 알아만 둬라 C라는 값을 크게 하면 ~~ 
# grid_search = GridSearchCV(svm, param_grid, cv=3) #하이퍼파라미터 서치할 모델 설정, 각 모델별로 5-cross validation 실행
# grid_search.fit(train_vectors, train_data['label'])  # transformed train data로 학습

# print(grid_search.cv_results_['mean_test_score']) # 파라미터 서치 결과
# print(grid_search.best_params_) # 최고의 모델 파라미터

# #파라미터 서치 결과 출력

- 그리드 서치가 정확히 뭐지?
- 3번 검증했을 때 9개의 성능으로 나온다.
c가 1이고 max가 500일 때 최고의 하이퍼 파라미터 성능이더라
- grid search cv가 무슨 값이라고 하셨지? - 교차검증을 몇 번 돌릴지 5-cross이면 cv=5 이런 거 같다!
그래서 5번의 값이 나오고 그것의 평균도 구하고 한다


In [42]:
from sklearn.metrics import accuracy_score # Accuracy 측정 함수 import

# final_model = grid_search.best_estimator_ # grid search에 의해 정해진 제일 좋은 하이퍼파라미터
final_model = svm

pred_results = final_model.predict(test_vectors) # 최종 모델로 test 데이터 예측
accuracy = accuracy_score(test_data['label'], pred_results) # 정확도 측정

print("Accuracy: {:.2f}%".format(accuracy*100)) #정확도 출력

Accuracy: 82.30%
